In [94]:
import numpy as np
import pandas 
from elasticsearch import Elasticsearch
import json

In [95]:
elastic_client = Elasticsearch()


In [115]:
response = elastic_client.search( index='dossier1', body={} )

total_docs = response["hits"]["total"]["value"]
response = elastic_client.search(
    index='dossier1',
    body={},
    size=total_docs
)
elastic_docs = response["hits"]["hits"]
"""for key, val in response["hits"].items():
    if key == "hits":
        for num, doc in enumerate(val):
            print (num, '-->', doc, "\n")"""
fields = {}
for num, doc in enumerate(elastic_docs):
    pass
    # do stuff here!
    # get source data from document
    source_data = doc["_source"]
    # iterate source data (use iteritems() for Python 2)
    for key, val in source_data.items():
        try:
            fields[key] = np.append(fields[key], val)
        except KeyError:
            fields[key] = np.array([val])


    
elastic_df = pandas.DataFrame(fields)


df=elastic_df

np.random.shuffle(df.values)
df

,nombre_de_signature,id_procedure,journee,dossier_en_cours,nom_procedure,nombre_de_ci,poid_moyen,@version,qte_moyenne,nombre_de_dossier,@timestamp,good_hs_code,nombre_de_rejection,type,duree_moyenne
0,1,cte,2018-05-14T22:00:00.000Z,3,Certificat contrôle technique export,0,None,1,5,4,2020-06-03T22:10:28.943Z,44034915000,0,dossier1,2.796118e+06
1,1,empotage,2018-07-20T22:00:00.000Z,3,Certificat d'empotage,1,None,1,100,4,2020-06-03T22:10:28.948Z,18010012000,0,dossier1,1.179983e+08
2,0,expec,2018-08-29T22:00:00.000Z,3,E-FORCE : Circuit de prise en charge à l'EXPORT,0,None,1,1800,3,2020-06-03T22:10:28.955Z,85366930903,0,dossier1,2.310667e+03
3,1,cte,2018-09-21T22:00:00.000Z,0,Certificat contrôle technique export,0,None,1,100,1,2020-06-03T22:10:28.960Z,18010012000,0,dossier1,1.902815e+09
4,0,bacm,2018-10-03T22:00:00.000Z,1,Paiement de la taxe de vérification SGS,0,None,1,None,1,2020-06-03T22:10:28.965Z,None,0,dossier1,4.482463e+07
5,0,bacm,2018-10-06T22:00:00.000Z,4,Paiement de la taxe de vérification SGS,0,None,1,None,4,2020-06-03T22:10:28.970Z,None,0,dossier1,9.552544e+05
6,0,bacm,2018-10-08T22:00:00.000Z,3,Paiement de la taxe de vérification SGS,0,None,1,None,3,2020-06-03T22:10:28.973Z,None,0,dossier1,1.388727e+05
7,1,cte,2018-10-15T22:00:00.000Z,0,Certificat contrôle technique export,0,None,1,1,1,2020-06-03T22:10:28.977Z,18010012000,0,dossier1,2.087958e+06
8,0,expec,2018-09-02T22:00:00.000Z,3,E-FORCE : Circuit de prise en charge à l'EXPORT,0,None,1,600.667,3,2020-06-03T22:10:28.957Z,85366930903,0,dossier1,7.976667e+02
9,1,cte,2018-05-29T22:00:00.000Z,0,Certificat contrôle technique export,0,None,1,5,1,2020-06-03T22:10:28.946Z,44034915000,0,dossier1,8.708897e+06


# Nettoyage des données 


In [116]:
import tensorflow as tf
tf.random.set_seed(123)
from IPython.display import clear_output
from matplotlib import pyplot as plt
import cx_Oracle


extraction du vocabulaire

In [117]:
#à ne pas executer no more necessary

dftrain=elastic_df
import cx_Oracle

#connection = cx_Oracle.connect("orchestra", "orchestra", "192.168.56.3/orcl")


class OraExec(object):
    _db_connection = None
    _db_cur = None

    def __init__(self):
        self._db_connection = cx_Oracle.connect('orchestra/orchestra@192.168.56.3:1521/GUCE')
        self._db_cur = self._db_connection.cursor()
    #con=OraExec()
    def getProcedures(self):
        #ver = self._db_connection.version.split(".")
        #print (ver)
        self._db_cur.execute("select ID from orchestra_process ")

       
        return self._db_cur

    
con=OraExec()
cur=con.getProcedures()
voc_procedures=[]
for result in cur:
    voc_procedures.append(result[0])
#len(voc_procedures)
    

DatabaseError: DPI-1047: Cannot locate a 64-bit Oracle Client library: "The specified module could not be found". See https://oracle.github.io/odpi/doc/installation.html#windows for help

ici on fournit les données de facons conforme: 
    on doe tout les features , et on donnes les bons types à nos column pour qu'elle puisse adhérer avec l'algorithme de onehotencoding que nous fournis tensorflow

In [118]:
df=elastic_df

df["journee"] = pandas.to_datetime(df["journee"])
df["good_hs_code"] = df["good_hs_code"].astype(str) 

#dftrain.dtypes

df["day"] = df["journee"].dt.day
df["month"] = df["journee"].dt.month
df["year"] = df["journee"].dt.year
df["weekday"] = df["journee"].dt.weekday
df=df.drop(["@version","@timestamp","journee","type"],axis=1)
df.fillna(value=pandas.np.nan, inplace=True)


In [123]:

from sklearn.model_selection import train_test_split

train, test = train_test_split(elastic_df, test_size=0.2)
y=elastic_df['duree_moyenne']

#deprecate
print(len(df ))
print(len(train))
print(len(test))

y_train=(train['duree_moyenne']).to_numpy()
y_test=(test['duree_moyenne']).to_numpy()

df_train=train[['nombre_de_signature', 'dossier_en_cours',
                'nombre_de_ci','poid_moyen','qte_moyenne','nombre_de_dossier'
                  ,'nombre_de_rejection','day','month',
                'year','weekday','id_procedure', 'nom_procedure' ,'good_hs_code']]
df_test=test[['nombre_de_signature', 'dossier_en_cours',
                'nombre_de_ci','poid_moyen','qte_moyenne','nombre_de_dossier'
                  ,'nombre_de_rejection','day','month',
                'year','weekday','id_procedure', 'nom_procedure' ,'good_hs_code']]
data=df_train.to_numpy()
df_train = pandas.DataFrame({'nombre_de_signature': data[:, 0], 'dossier_en_cours': data[:, 1],
                'nombre_de_ci' : data[:, 2],'poid_moyen' : data[:, 3],'qte_moyenne' : data[:, 4],
                         'nombre_de_dossier' : data[:, 5]
                  ,'nombre_de_rejection' : data[:, 6],'day' : data[:, 7],'month' : data[:, 8],
                'year'  : data[:, 9] ,'weekday' : data[:, 10],'id_procedure' : data[:, 11],
                         'nom_procedure' : data[:, 12] ,'good_hs_code' : data[:, 13]})
dftrain

255
204
51


,nombre_de_signature,id_procedure,journee,dossier_en_cours,nom_procedure,nombre_de_ci,poid_moyen,@version,qte_moyenne,nombre_de_dossier,@timestamp,good_hs_code,nombre_de_rejection,type,duree_moyenne,day,month,year,weekday
0,1,cte,2018-05-14 22:00:00+00:00,3,Certificat contrôle technique export,0,None,1,5,4,2020-06-03T22:10:28.943Z,44034915000,0,dossier1,2.796118e+06,14,5,2018,0
1,1,empotage,2018-07-20 22:00:00+00:00,3,Certificat d'empotage,1,None,1,100,4,2020-06-03T22:10:28.948Z,18010012000,0,dossier1,1.179983e+08,20,7,2018,4
2,0,expec,2018-08-29 22:00:00+00:00,3,E-FORCE : Circuit de prise en charge à l'EXPORT,0,None,1,1800,3,2020-06-03T22:10:28.955Z,85366930903,0,dossier1,2.310667e+03,29,8,2018,2
3,1,cte,2018-09-21 22:00:00+00:00,0,Certificat contrôle technique export,0,None,1,100,1,2020-06-03T22:10:28.960Z,18010012000,0,dossier1,1.902815e+09,21,9,2018,4
4,0,bacm,2018-10-03 22:00:00+00:00,1,Paiement de la taxe de vérification SGS,0,None,1,None,1,2020-06-03T22:10:28.965Z,None,0,dossier1,4.482463e+07,3,10,2018,2
5,0,bacm,2018-10-06 22:00:00+00:00,4,Paiement de la taxe de vérification SGS,0,None,1,None,4,2020-06-03T22:10:28.970Z,None,0,dossier1,9.552544e+05,6,10,2018,5
6,0,bacm,2018-10-08 22:00:00+00:00,3,Paiement de la taxe de vérification SGS,0,None,1,None,3,2020-06-03T22:10:28.973Z,None,0,dossier1,1.388727e+05,8,10,2018,0
7,1,cte,2018-10-15 22:00:00+00:00,0,Certificat contrôle technique export,0,None,1,1,1,2020-06-03T22:10:28.977Z,18010012000,0,dossier1,2.087958e+06,15,10,2018,0
8,0,expec,2018-09-02 22:00:00+00:00,3,E-FORCE : Circuit de prise en charge à l'EXPORT,0,None,1,600.667,3,2020-06-03T22:10:28.957Z,85366930903,0,dossier1,7.976667e+02,2,9,2018,6
9,1,cte,2018-05-29 22:00:00+00:00,0,Certificat contrôle technique export,0,None,1,5,1,2020-06-03T22:10:28.946Z,44034915000,0,dossier1,8.708897e+06,29,5,2018,1


In [126]:
CATEGORICAL_COLUMNS = ['id_procedure', 'nom_procedure' ,'good_hs_code']

                       #journee
NUMERIC_COLUMNS = ['nombre_de_signature', 'dossier_en_cours','nombre_de_ci','poid_moyen','qte_moyenne','nombre_de_dossier'
                  ,'nombre_de_rejection','day','month','year','weekday']

def one_hot_cat_column(feature_name, vocab):
    return tf.feature_column.indicator_column(
      tf.feature_column.categorical_column_with_vocabulary_list(feature_name,
                                                 vocab))
feature_columns = []
for feature_name in CATEGORICAL_COLUMNS:
    # Need to one-hot encode categorical features.
    vocabulary = df_train[feature_name].unique()
    feature_columns.append(one_hot_cat_column(feature_name, vocabulary))

for feature_name in NUMERIC_COLUMNS:
    feature_columns.append(tf.feature_column.numeric_column(feature_name,
                                           dtype=tf.float32))
example = dict(df)
example_train = dict(df_train)

voc_procedures = dftrain["id_procedure"].unique()

class_fc = tf.feature_column.indicator_column(tf.feature_column.categorical_column_with_vocabulary_list(
    'id_procedure',voc_procedures))
#print(class_fc)

#print('Feature value: "{}"'.format(example['id_procedure'].iloc[0]))
#print('One-hot encoded: ', tf.keras.layers.DenseFeatures([class_fc])(example).numpy())
tf.keras.layers.DenseFeatures(feature_columns)(example).numpy()
tf.keras.layers.DenseFeatures(feature_columns)(example_train).numpy()

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type int).

In [ ]:
#normalisation des données en batch 
NUM_EXAMPLES = int(len(y_train)/2)

def make_input_fn(X, y, n_epochs=None, shuffle=True):
    def input_fn():
        dataset = tf.data.Dataset.from_tensor_slices((dict(X), y))
        if shuffle:
          dataset = dataset.shuffle(NUM_EXAMPLES)
        # For training, cycle thru dataset as many times as need (n_epochs=None).
        dataset = dataset.repeat(n_epochs)
        # In memory training doesn't use batching.
        dataset = dataset.batch(NUM_EXAMPLES)
        return dataset
    return input_fn

# Training and evaluation input functions.
train_input_fn = make_input_fn(df_train, y_train)
test_input_fn = make_input_fn(df_test, y_test, shuffle=False, n_epochs=1)
dataset=train_input_fn()
dataset

linear_est = tf.estimator.LinearClassifier(feature_columns)

# Train model.
linear_est.train(train_input_fn, max_steps=100)

# Evaluation.
result = linear_est.evaluate(eval_input_fn)
clear_output()
print(pd.Series(result))

In [125]:

from sklearn.ensemble import RandomForestRegressor
regressor =RandomForestRegressor(n_estimators=10, random_state=0)
#regressor.fit(X.reshape(-1,1), y.to_numpy())
np.where(np.isnan(X)) 

(array([  0,   1,   2,   3,   4,   4,   5,   5,   6,   6,   7,   8,   9,
         10,  11,  12,  13,  13,  14,  14,  15,  17,  18,  19,  19,  20,
         20,  21,  21,  22,  22,  23,  23,  24,  24,  25,  25,  26,  26,
         27,  28,  29,  30,  31,  31,  32,  32,  33,  33,  34,  35,  35,
         36,  37,  37,  38,  39,  40,  40,  41,  41,  42,  42,  44,  44,
         45,  45,  48,  49,  49,  50,  50,  51,  51,  52,  53,  54,  54,
         55,  55,  56,  56,  57,  58,  58,  59,  60,  61,  61,  62,  62,
         63,  63,  64,  64,  65,  66,  66,  67,  67,  68,  68,  69,  69,
         70,  70,  73,  73,  75,  75,  76,  76,  77,  79,  79,  80,  81,
         82,  82,  83,  83,  84,  85,  86,  87,  90,  90,  91,  91,  92,
         92,  93,  93,  94,  94,  95,  95,  96,  96,  97,  98,  99,  99,
        100, 100, 101, 101, 102, 102, 103, 103, 104, 104, 106, 106, 107,
        108, 109, 110, 111, 111, 112, 112, 113, 114, 115, 115, 116, 116,
        117, 117, 118, 118, 120, 121, 121, 122, 122

In [ ]:

x_grid=np.arange(min())

In [31]:
type(df_ohe)

numpy.ndarray

(array([  0,   1,   2,   3,   4,   4,   5,   5,   6,   6,   7,   8,   9,
         10,  11,  12,  13,  13,  14,  14,  15,  17,  18,  19,  19,  20,
         20,  21,  21,  22,  22,  23,  23,  24,  24,  25,  25,  26,  26,
         27,  28,  29,  30,  31,  31,  32,  32,  33,  33,  34,  35,  35,
         36,  37,  37,  38,  39,  40,  40,  41,  41,  42,  42,  44,  44,
         45,  45,  48,  49,  49,  50,  50,  51,  51,  52,  53,  54,  54,
         55,  55,  56,  56,  57,  58,  58,  59,  60,  61,  61,  62,  62,
         63,  63,  64,  64,  65,  66,  66,  67,  67,  68,  68,  69,  69,
         70,  70,  73,  73,  75,  75,  76,  76,  77,  79,  79,  80,  81,
         82,  82,  83,  83,  84,  85,  86,  87,  90,  90,  91,  91,  92,
         92,  93,  93,  94,  94,  95,  95,  96,  96,  97,  98,  99,  99,
        100, 100, 101, 101, 102, 102, 103, 103, 104, 104, 106, 106, 107,
        108, 109, 110, 111, 111, 112, 112, 113, 114, 115, 115, 116, 116,
        117, 117, 118, 118, 120, 121, 121, 122, 122